In [ ]:
# Import all the stuff we need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report
import xgboost as xgb
import shap
import warnings
warnings.filterwarnings('ignore')  # suppress annoying warnings

# random seed so we get consistent results
np.random.seed(42)
# actually let me double check this works
print(f"Random seed set to 42. Test random number: {np.random.rand()}")

# Configure plotting - I like the seaborn style better
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
# make text a bit bigger so I can actually read it
plt.rcParams['font.size'] = 11

print("All libraries loaded successfully!")
print("Ready to start the churn prediction analysis")


In [ ]:
def generate_telecom_data(num_customers=10000):
    """
    Generate fake telecom customer data that looks realistic
    
    Parameters:
    num_customers (int): how many customers to create
    
    Returns:
    pd.DataFrame: our synthetic dataset
    """
    print(f"Creating {num_customers:,} fake customers...")
    
    # let's build this step by step
    data = {}
    
    # customer IDs - make them look real
    data['customer_id'] = [f'CUST_{i+1:06d}' for i in range(num_customers)]
    
    # age distribution - most people are middle aged
    data['age'] = np.random.normal(45, 15, num_customers).astype(int)
    
    # gender split
    data['gender'] = np.random.choice(['Male', 'Female'], num_customers)
    
    # tenure - exponential makes sense, newer customers churn more
    data['tenure_months'] = np.random.exponential(20, num_customers).astype(int)
    
    # contract types - most people go month to month unfortunately
    data['contract_type'] = np.random.choice(['month-to-month', 'one-year', 'two-year'], 
                                            num_customers, p=[0.5, 0.3, 0.2])
    
    # internet service types
    data['internet_service'] = np.random.choice(['DSL', 'Fiber', 'None'], 
                                               num_customers, p=[0.4, 0.4, 0.2])
    
    # tech support - most people don't have it
    data['tech_support'] = np.random.choice(['Yes', 'No'], num_customers, p=[0.3, 0.7])
    
    # streaming services - getting more popular
    data['streaming_services'] = np.random.choice(['Yes', 'No'], num_customers, p=[0.4, 0.6])
    
    # payment methods
    payment_options = ['Electronic check', 'Mailed check', 'Bank transfer', 'Credit card']
    data['payment_method'] = np.random.choice(payment_options, 
                                             num_customers, p=[0.35, 0.2, 0.2, 0.25])
    
    # fix age range - nobody under 18 or over 80
    data['age'] = np.clip(data['age'], 18, 80)
    
    # tenure should be reasonable too
    data['tenure_months'] = np.clip(data['tenure_months'], 1, 72)
    
    # now for monthly charges - this depends on service type
    monthly_charges_list = []
    for service_type in data['internet_service']:
        if service_type == 'None':
            # basic service is cheaper
            charge = np.random.normal(35, 10)
        elif service_type == 'DSL':
            # DSL is mid-range
            charge = np.random.normal(55, 15)
        else:  # Fiber
            # fiber is expensive but fast
            charge = np.random.normal(85, 20)
        
        # keep charges reasonable
        charge = max(20, min(120, charge))
        monthly_charges_list.append(charge)
    
    data['monthly_charges'] = [round(x, 2) for x in monthly_charges_list]
    
    # total charges should correlate with tenure and monthly charges
    # adding some noise to make it realistic
    total_charges_calc = []
    for i in range(num_customers):
        base_total = data['tenure_months'][i] * data['monthly_charges'][i]
        # add some randomness
        noise = np.random.normal(0, 100)
        total = base_total + noise
        total_charges_calc.append(max(0, round(total, 2)))  # can't be negative
    
    data['total_charges'] = total_charges_calc
    
    # now the tricky part - generate realistic churn patterns
    # this is the most important part to get right
    churn_probabilities = []
    
    for i in range(num_customers):
        # start with base churn rate
        churn_prob = 0.15
        
        # contract type is huge - month-to-month customers churn way more
        contract = data['contract_type'][i]
        if contract == 'month-to-month':
            churn_prob += 0.25  # these people leave all the time
        elif contract == 'one-year':
            churn_prob += 0.1
        # two-year contracts are pretty sticky
        
        # tenure matters a lot - new customers leave more
        tenure = data['tenure_months'][i]
        if tenure < 6:
            churn_prob += 0.2  # new customers are risky
        elif tenure < 12:
            churn_prob += 0.1
        elif tenure > 24:
            churn_prob -= 0.1  # loyal customers
        
        # price sensitivity
        monthly_charge = data['monthly_charges'][i]
        if monthly_charge > 90:
            churn_prob += 0.15  # expensive plans drive churn
        elif monthly_charge < 30:
            churn_prob += 0.1  # but so do cheap ones (probably price shoppers)
        
        # tech support helps retention
        if data['tech_support'][i] == 'Yes':
            churn_prob -= 0.08  # good support keeps customers
        
        # payment method matters too
        if data['payment_method'][i] == 'Electronic check':
            churn_prob += 0.1  # these customers are flighty
        
        # age patterns
        age = data['age'][i]
        if age < 25 or age > 65:
            churn_prob += 0.05  # young people switch a lot, old people get confused
        
        # make sure probability is reasonable
        churn_prob = max(0.01, min(0.8, churn_prob))
        churn_probabilities.append(churn_prob)
    
    # actually generate the churn flags
    data['churn'] = np.random.binomial(1, churn_probabilities, num_customers)
    
    # turn into dataframe
    df = pd.DataFrame(data)
    
    print(f"Done! Created {len(df):,} customers")
    print(f"Dataset dimensions: {df.shape}")
    print(f"Overall churn rate: {df['churn'].mean():.2%}")
    
    return df

# let's generate our dataset
telecom_data = generate_telecom_data(10000)

# take a look at what we created
print("\n" + "="*50)
print("DATASET OVERVIEW")
print("="*50)
print(telecom_data.head())


In [ ]:
# let's check if our data looks good
print("DATA QUALITY CHECK")
print("="*40)
print(f"We have {len(telecom_data):,} rows and {len(telecom_data.columns)} columns")
print(f"Shape: {telecom_data.shape}")
print()

# check data types
print("What types of data do we have:")
print(telecom_data.dtypes)
print()

# any missing values? (shouldn't be any since we generated it)
print("Missing values check:")
missing_vals = telecom_data.isnull().sum()
if missing_vals.sum() > 0:
    print(missing_vals[missing_vals > 0])
else:
    print("No missing values - good!")
print()

# basic stats
print("Quick stats on the numeric columns:")
print(telecom_data.describe())
print()

# let's look at our categorical variables
print("Distribution of categorical features:")
cat_cols = ['gender', 'contract_type', 'internet_service', 'tech_support', 
           'streaming_services', 'payment_method']

for col in cat_cols:
    print(f"\n{col.upper()}:")
    counts = telecom_data[col].value_counts()
    print(counts)
    print(f"({counts.nunique()} unique values)")


In [ ]:
# make a copy so we don't mess up the original
df_working = telecom_data.copy()

print("GETTING DATA READY FOR ML")
print("="*40)

# split into features (X) and target (y)
# don't need customer_id for modeling
X = df_working.drop(['customer_id', 'churn'], axis=1)
y = df_working['churn']

print(f"Features: {X.shape}")
print(f"Target variable distribution:")
print(y.value_counts(normalize=True))
print()

# figure out which columns are which type
numeric_cols = ['age', 'tenure_months', 'monthly_charges', 'total_charges']
categorical_cols = ['gender', 'contract_type', 'internet_service', 'tech_support', 
                   'streaming_services', 'payment_method']

print(f"Numeric features: {numeric_cols}")
print(f"Categorical features: {categorical_cols}")
print()

# need to encode the categorical stuff for sklearn
encoders = {}  # store encoders in case we need them later
X_encoded = X.copy()

print("Converting categorical variables to numbers...")
for col in categorical_cols:
    encoder = LabelEncoder()
    X_encoded[col] = encoder.fit_transform(X[col])
    encoders[col] = encoder
    print(f"  {col}: {len(encoder.classes_)} categories -> {encoder.classes_}")

print("\nSample of encoded data:")
print(X_encoded.head())
print()

# scale the numeric features so they're all on similar scales
print("Scaling numeric features...")
scaler = StandardScaler()
X_final = X_encoded.copy()
X_final[numeric_cols] = scaler.fit_transform(X_encoded[numeric_cols])

print("Scaling info:")
for i, col in enumerate(numeric_cols):
    print(f"  {col}: mean={scaler.mean_[i]:.2f}, std={scaler.scale_[i]:.2f}")

print("\nScaled data sample:")
print(X_final[numeric_cols].head())


In [ ]:
# split into train and test sets
# using stratify to keep the same churn ratio in both sets
X_train, X_test, y_train, y_test = train_test_split(
    X_final, y, 
    test_size=0.2,  # 80-20 split is pretty standard
    random_state=42, 
    stratify=y  # this keeps proportions balanced
)

print("TRAIN-TEST SPLIT")
print("="*30)
print(f"Training data: {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X_final)*100:.0f}%)")
print(f"Test data: {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X_final)*100:.0f}%)")
print()

print("Checking that churn rates are similar:")
train_churn_rate = y_train.mean()
test_churn_rate = y_test.mean()
print(f"Training set churn: {train_churn_rate:.3f}")
print(f"Test set churn: {test_churn_rate:.3f}")
print("Good - they're about the same!")
print()

print("Final feature list:")
feature_list = list(X_train.columns)
print(feature_list)
print(f"Total features: {len(feature_list)}")


In [ ]:
# let's understand the churn patterns before we build models
print("UNDERSTANDING THE CHURN PROBLEM")
print("="*40)

# basic churn stats
churn_rate = telecom_data['churn'].mean()
total_customers = len(telecom_data)
churned_count = telecom_data['churn'].sum()
retained_count = total_customers - churned_count

print(f"Overall churn rate: {churn_rate:.1%}")
print(f"Total customers: {total_customers:,}")
print(f"Lost customers: {churned_count:,}")
print(f"Kept customers: {retained_count:,}")
print()

# contract type is probably the biggest driver
print("How does contract type affect churn?")
print("-" * 35)
contract_analysis = telecom_data.groupby('contract_type')['churn'].agg(['count', 'sum', 'mean'])
contract_analysis.columns = ['Total', 'Churned', 'Rate']
contract_analysis['Percentage'] = (contract_analysis['Rate'] * 100).round(1)

print(contract_analysis)
print("Wow - month-to-month customers churn way more!")
print()

# let's check other important factors
interesting_features = ['internet_service', 'tech_support', 'payment_method']

for feature in interesting_features:
    print(f"\nWhat about {feature.replace('_', ' ')}?")
    print("-" * 25)
    feature_stats = telecom_data.groupby(feature)['churn'].agg(['count', 'mean'])
    feature_stats.columns = ['Count', 'Churn_Rate']
    feature_stats['Churn_Pct'] = (feature_stats['Churn_Rate'] * 100).round(1)
    # sort by churn rate to see the worst first
    feature_stats = feature_stats.sort_values('Churn_Rate', ascending=False)
    print(feature_stats)
    
    # add some commentary
    if feature == 'payment_method':
        highest_churn = feature_stats.index[0]
        print(f"Electronic check customers are the riskiest!")
    elif feature == 'tech_support':
        print("Having tech support definitely helps retention")
    elif feature == 'internet_service':
        print("Fiber customers churn more - probably price sensitive")
    print()


In [ ]:
# let's make some charts to visualize the patterns
# I like to see everything at once
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Customer Churn Patterns - The Visual Story', fontsize=16, y=1.02)

# overall split between churned and retained
churn_counts = telecom_data['churn'].value_counts()
axes[0, 0].pie(churn_counts.values, labels=['Stayed', 'Left'], autopct='%1.1f%%', 
               colors=['lightgreen', 'lightcoral'])  # green=good, red=bad
axes[0, 0].set_title('Overall: Who Stayed vs Left')

# contract type differences - this should be dramatic
contract_churn_rates = telecom_data.groupby('contract_type')['churn'].mean() * 100
contract_churn_rates.plot(kind='bar', ax=axes[0, 1], color='steelblue', rot=45)
axes[0, 1].set_title('Churn % by Contract Length')
axes[0, 1].set_ylabel('Churn Rate (%)')
axes[0, 1].grid(axis='y', alpha=0.3)

# tenure patterns - let's group customers by how long they've been with us
data_copy = telecom_data.copy()  # don't want to modify original
data_copy['tenure_group'] = pd.cut(data_copy['tenure_months'], 
                                  bins=[0, 12, 24, 36, 72], 
                                  labels=['0-12 mo', '13-24 mo', '25-36 mo', '37+ mo'])
tenure_churn_rates = data_copy.groupby('tenure_group')['churn'].mean() * 100
tenure_churn_rates.plot(kind='bar', ax=axes[0, 2], color='orange', rot=0)
axes[0, 2].set_title('Churn by How Long They\'ve Been Customers')
axes[0, 2].set_ylabel('Churn Rate (%)')
axes[0, 2].grid(axis='y', alpha=0.3)

# how do monthly charges look for churned vs retained customers?
churned_customers = telecom_data[telecom_data['churn'] == 1]['monthly_charges']
staying_customers = telecom_data[telecom_data['churn'] == 0]['monthly_charges']
axes[1, 0].hist([staying_customers, churned_customers], bins=25, alpha=0.6, 
               label=['Stayed', 'Left'], color=['green', 'red'])
axes[1, 0].set_title('Monthly Charges: Stayers vs Leavers')
axes[1, 0].set_xlabel('Monthly Bill ($)')
axes[1, 0].set_ylabel('Number of Customers')
axes[1, 0].legend()
axes[1, 0].grid(axis='y', alpha=0.3)

# internet service type patterns
internet_churn_pct = telecom_data.groupby('internet_service')['churn'].mean() * 100
internet_churn_pct.plot(kind='bar', ax=axes[1, 1], color='purple', rot=30)
axes[1, 1].set_title('Internet Service Type vs Churn')
axes[1, 1].set_ylabel('Churn Rate (%)')
axes[1, 1].grid(axis='y', alpha=0.3)

# payment method - this one is interesting
payment_churn_pct = telecom_data.groupby('payment_method')['churn'].mean() * 100
payment_churn_pct.plot(kind='bar', ax=axes[1, 2], color='teal', rot=30)
axes[1, 2].set_title('Payment Method Impact on Churn')
axes[1, 2].set_ylabel('Churn Rate (%)')
axes[1, 2].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# let's also look at correlations to see what matters most
print("\nWHICH FEATURES CORRELATE WITH CHURN?")
print("="*35)
# use the encoded data for correlations
correlations = X_encoded.corrwith(y).sort_values(key=abs, ascending=False)
print("Features ranked by correlation with churn (absolute value):")
for feature, corr in correlations.items():
    print(f"  {feature}: {corr:.3f}")
print("\nHigher absolute values = stronger relationship with churn")


In [ ]:
# time for some machine learning! starting with logistic regression
print("BUILDING OUR FIRST MODEL - LOGISTIC REGRESSION")
print("="*50)

# logistic regression is a good baseline - simple and interpretable
log_reg = LogisticRegression(random_state=42, max_iter=1000)  # increase max_iter to avoid convergence warnings
print("Training logistic regression model...")
log_reg.fit(X_train, y_train)
print("Done!")

# make predictions on test set
print("Making predictions...")
lr_predictions = log_reg.predict(X_test)
lr_probabilities = log_reg.predict_proba(X_test)[:, 1]  # probability of churn

# how did we do?
print("\nModel Performance:")
print("-" * 20)
acc = accuracy_score(y_test, lr_predictions)
precision = precision_score(y_test, lr_predictions)
recall = recall_score(y_test, lr_predictions)
f1 = f1_score(y_test, lr_predictions)

print(f"Accuracy: {acc:.3f} ({acc*100:.1f}%)")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1-Score: {f1:.3f}")

# AUC is really important for churn prediction
fpr, tpr, _ = roc_curve(y_test, lr_probabilities)
auc_score = auc(fpr, tpr)
print(f"AUC Score: {auc_score:.3f}")
print()

# confusion matrix tells the whole story
print("Confusion Matrix (how many we got right/wrong):")
cm = confusion_matrix(y_test, lr_predictions)
print(cm)
print("Format: [[True Neg, False Pos], [False Neg, True Pos]]")
print()

# detailed breakdown
print("Detailed Results:")
print(classification_report(y_test, lr_predictions, target_names=['Stayed', 'Churned']))

# what features matter most?
feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': log_reg.coef_[0],
    'Abs_Coeff': np.abs(log_reg.coef_[0])
}).sort_values('Abs_Coeff', ascending=False)

print("Most Important Features (top 10):")
print(feature_importance.head(10))

# store these results for later comparison
lr_results = {
    'model': log_reg,
    'accuracy': acc,
    'auc': auc_score,
    'predictions': lr_predictions,
    'probabilities': lr_probabilities
}


In [ ]:
# now let's try XGBoost - this should perform better
print("TRYING XGBOOST - THE FANCY GRADIENT BOOSTING MODEL")
print("="*55)

# XGBoost usually works better than logistic regression for this kind of problem
# but first we need to tune the hyperparameters

# let's try a few different parameter combinations
# keeping it simple since full grid search takes forever
params_to_try = {
    'n_estimators': [100, 200],  # how many trees
    'max_depth': [3, 5, 7],      # how deep each tree goes
    'learning_rate': [0.01, 0.1, 0.2],  # how fast it learns
    'subsample': [0.8, 1.0]      # what fraction of data to use per tree
}

print("Setting up XGBoost model...")
xgb_model = xgb.XGBClassifier(random_state=42, eval_metric='logloss')

print("Starting hyperparameter search - this might take a while...")
print("(Going to try different combinations and pick the best one)")

# use grid search to find best parameters
# using 3-fold CV to keep it reasonable
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=params_to_try,
    scoring='roc_auc',  # AUC is what we care about most
    cv=3,
    n_jobs=-1,  # use all CPU cores
    verbose=1   # show progress
)

# this is the time-consuming part
grid_search.fit(X_train, y_train)

# what did we find?
best_params = grid_search.best_params_
best_cv_score = grid_search.best_score_

print(f"\nBest parameters found: {best_params}")
print(f"Best cross-validation AUC: {best_cv_score:.4f}")
print("Not bad!")
print()

# now let's train the best model and see how it performs
print("Training the optimized XGBoost model...")
final_xgb = grid_search.best_estimator_

# test it on our holdout data
print("Making predictions on test set...")
xgb_predictions = final_xgb.predict(X_test)
xgb_probabilities = final_xgb.predict_proba(X_test)[:, 1]

# calculate all the metrics
xgb_acc = accuracy_score(y_test, xgb_predictions)
xgb_precision = precision_score(y_test, xgb_predictions)
xgb_recall = recall_score(y_test, xgb_predictions)
xgb_f1 = f1_score(y_test, xgb_predictions)

print("\nXGBOOST RESULTS")
print("="*20)
print(f"Accuracy: {xgb_acc:.3f} ({xgb_acc*100:.1f}%)")
print(f"Precision: {xgb_precision:.3f}")
print(f"Recall: {xgb_recall:.3f}")
print(f"F1-Score: {xgb_f1:.3f}")

# most important - the AUC score
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, xgb_probabilities)
xgb_auc = auc(fpr_xgb, tpr_xgb)
print(f"AUC Score: {xgb_auc:.3f}")
print()

# confusion matrix
print("Confusion Matrix:")
xgb_cm = confusion_matrix(y_test, xgb_predictions)
print(xgb_cm)
print()

# detailed results
print("Full Classification Report:")
print(classification_report(y_test, xgb_predictions, target_names=['Stayed', 'Churned']))

# how much better is this than logistic regression?
improvement = ((xgb_auc - auc_score) / auc_score) * 100
print(f"\nCOMPARISON TO BASELINE:")
print(f"Logistic Regression AUC: {auc_score:.3f}")
print(f"XGBoost AUC: {xgb_auc:.3f}")
print(f"Improvement: {improvement:.1f}% better!")

# save these results too
xgb_results = {
    'model': final_xgb,
    'accuracy': xgb_acc,
    'auc': xgb_auc,
    'predictions': xgb_predictions,
    'probabilities': xgb_probabilities
}


In [ ]:
# let's visualize how the two models compare
plt.figure(figsize=(10, 8))

# plot both ROC curves
plt.plot(fpr, tpr, label=f'Logistic Regression (AUC = {auc_score:.3f})', 
         linewidth=2.5, color='blue')
plt.plot(fpr_xgb, tpr_xgb, label=f'XGBoost (AUC = {xgb_auc:.3f})', 
         linewidth=2.5, color='red')

# add the "random guessing" line for reference
plt.plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Random Guessing (AUC = 0.500)')

# make it look nice
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - Specificity)', fontsize=12)
plt.ylabel('True Positive Rate (Sensitivity)', fontsize=12)
plt.title('ROC Curves: Logistic Regression vs XGBoost', fontsize=14, pad=20)
plt.legend(loc="lower right", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# create a nice comparison table
print("SIDE-BY-SIDE MODEL COMPARISON")
print("="*35)

results_comparison = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC'],
    'Logistic Reg': [acc, precision, recall, f1, auc_score],
    'XGBoost': [xgb_acc, xgb_precision, xgb_recall, xgb_f1, xgb_auc]
})

# calculate improvements
results_comparison['Improvement'] = ((results_comparison['XGBoost'] - results_comparison['Logistic Reg']) / 
                                   results_comparison['Logistic Reg'] * 100)

# round for display
results_comparison['Logistic Reg'] = results_comparison['Logistic Reg'].round(3)
results_comparison['XGBoost'] = results_comparison['XGBoost'].round(3)
results_comparison['Improvement'] = results_comparison['Improvement'].round(1)

print(results_comparison.to_string(index=False))
avg_improvement = results_comparison['Improvement'].mean()
print(f"\nOverall average improvement: {avg_improvement:.1f}%")
print("XGBoost is clearly the winner!")


In [ ]:
# Feature Importance Analysis
print("FEATURE IMPORTANCE ANALYSIS")
print("="*40)

# XGBoost built-in feature importance
xgb_feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': best_xgb_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("Top 10 Most Important Features (XGBoost):")
print(xgb_feature_importance.head(10))
print()

# Visualize XGBoost feature importance
plt.figure(figsize=(12, 8))
top_features = xgb_feature_importance.head(10)
plt.barh(range(len(top_features)), top_features['Importance'], color='skyblue')
plt.yticks(range(len(top_features)), top_features['Feature'])
plt.xlabel('Feature Importance')
plt.title('Top 10 Feature Importance (XGBoost)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# SHAP Analysis
print("SHAP ANALYSIS")
print("="*20)
print("Computing SHAP values for model interpretability...")

# Create SHAP explainer
explainer = shap.TreeExplainer(best_xgb_model)
shap_values = explainer.shap_values(X_test[:1000])  # Use subset for efficiency

# SHAP summary plot
print("Generating SHAP summary plot...")
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_test[:1000], feature_names=X_train.columns, show=False)
plt.title('SHAP Feature Importance Summary')
plt.tight_layout()
plt.show()

# SHAP feature importance (mean absolute SHAP values)
shap_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'SHAP_Importance': np.abs(shap_values).mean(0)
}).sort_values('SHAP_Importance', ascending=False)

print("Top 10 Features by SHAP Importance:")
print(shap_importance.head(10))

# Combined feature importance comparison
plt.figure(figsize=(14, 8))

# Normalize importance scores for comparison
xgb_norm = xgb_feature_importance.set_index('Feature')['Importance'] / xgb_feature_importance['Importance'].max()
shap_norm = shap_importance.set_index('Feature')['SHAP_Importance'] / shap_importance['SHAP_Importance'].max()

# Get top 10 features from XGBoost
top_10_features = xgb_feature_importance.head(10)['Feature'].tolist()

# Create comparison plot
x = np.arange(len(top_10_features))
width = 0.35

plt.bar(x - width/2, [xgb_norm[f] for f in top_10_features], width, 
        label='XGBoost Importance', alpha=0.8, color='lightblue')
plt.bar(x + width/2, [shap_norm[f] for f in top_10_features], width, 
        label='SHAP Importance', alpha=0.8, color='lightcoral')

plt.xlabel('Features')
plt.ylabel('Normalized Importance')
plt.title('Feature Importance Comparison: XGBoost vs SHAP')
plt.xticks(x, top_10_features, rotation=45)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Business Impact Analysis
print("BUSINESS IMPACT ANALYSIS")
print("="*40)

# Get test set with predictions
test_results = X_test.copy()
test_results['actual_churn'] = y_test
test_results['churn_probability'] = y_pred_proba_xgb
test_results['predicted_churn'] = y_pred_xgb

# Add original customer data for revenue calculations
test_indices = y_test.index
test_results['monthly_charges'] = telecom_data.loc[test_indices, 'monthly_charges'].values
test_results['customer_id'] = telecom_data.loc[test_indices, 'customer_id'].values

# Identify top 10% high-risk customers
top_10_percent = int(len(test_results) * 0.1)
high_risk_customers = test_results.nlargest(top_10_percent, 'churn_probability')

print(f"Total test customers: {len(test_results):,}")
print(f"Top 10% high-risk customers: {len(high_risk_customers):,}")
print(f"High-risk threshold probability: {high_risk_customers['churn_probability'].min():.3f}")
print()

# Calculate current metrics for high-risk group
high_risk_actual_churn = high_risk_customers['actual_churn'].sum()
high_risk_churn_rate = high_risk_customers['actual_churn'].mean()
high_risk_monthly_revenue = high_risk_customers['monthly_charges'].sum()

print("HIGH-RISK CUSTOMER ANALYSIS")
print("-" * 30)
print(f"Actual churners in high-risk group: {high_risk_actual_churn:,}")
print(f"Churn rate in high-risk group: {high_risk_churn_rate:.1%}")
print(f"Monthly revenue from high-risk group: ${high_risk_monthly_revenue:,.2f}")
print()

# Business impact assumptions
retention_success_rate = 0.40  # Assume 40% of targeted customers can be retained
avg_customer_lifetime_months = 24  # Average customer lifetime
intervention_cost_per_customer = 50  # Cost to intervene per customer

# Calculate potential revenue saved
churners_in_high_risk = high_risk_customers[high_risk_customers['actual_churn'] == 1]
potential_saves = len(churners_in_high_risk) * retention_success_rate
monthly_revenue_saved = churners_in_high_risk['monthly_charges'].sum() * retention_success_rate
lifetime_revenue_saved = monthly_revenue_saved * avg_customer_lifetime_months

# Calculate intervention costs
total_intervention_cost = len(high_risk_customers) * intervention_cost_per_customer

# Calculate net benefit
net_benefit = lifetime_revenue_saved - total_intervention_cost

print("BUSINESS IMPACT SIMULATION")
print("-" * 35)
print(f"Retention success rate assumption: {retention_success_rate:.0%}")
print(f"Average customer lifetime: {avg_customer_lifetime_months} months")
print(f"Intervention cost per customer: ${intervention_cost_per_customer}")
print()
print(f"Potential customers saved: {potential_saves:.0f}")
print(f"Monthly revenue saved: ${monthly_revenue_saved:,.2f}")
print(f"Lifetime revenue saved: ${lifetime_revenue_saved:,.2f}")
print(f"Total intervention cost: ${total_intervention_cost:,.2f}")
print(f"Net benefit: ${net_benefit:,.2f}")
print(f"ROI: {(net_benefit / total_intervention_cost) * 100:.1f}%")
print()

# Scale to full customer base
full_customer_base = 10000
scaling_factor = full_customer_base / len(test_results)
scaled_net_benefit = net_benefit * scaling_factor

print("SCALED BUSINESS IMPACT (Full Customer Base)")
print("-" * 45)
print(f"Estimated annual net benefit: ${scaled_net_benefit * 12:,.2f}")
print(f"Estimated monthly net benefit: ${scaled_net_benefit:,.2f}")

# Create visualization of business impact
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Churn probability distribution
ax1.hist(test_results['churn_probability'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
ax1.axvline(high_risk_customers['churn_probability'].min(), color='red', linestyle='--', 
           label=f'Top 10% Threshold ({high_risk_customers["churn_probability"].min():.3f})')
ax1.set_xlabel('Churn Probability')
ax1.set_ylabel('Number of Customers')
ax1.set_title('Churn Probability Distribution')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Revenue impact breakdown
categories = ['Current\nMonthly Revenue', 'Revenue\nAt Risk', 'Potential\nSavings', 'Net Benefit\n(Lifetime)']
values = [high_risk_monthly_revenue, 
          churners_in_high_risk['monthly_charges'].sum(),
          monthly_revenue_saved,
          net_benefit / avg_customer_lifetime_months]

colors = ['lightblue', 'salmon', 'lightgreen', 'gold']
bars = ax2.bar(categories, values, color=colors, alpha=0.8)
ax2.set_ylabel('Revenue ($)')
ax2.set_title('Revenue Impact Analysis - High-Risk Customers')
ax2.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar, value in zip(bars, values):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
             f'${value:,.0f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()


In [ ]:
# Create comprehensive results dataset for export
print("CREATING EXPORT DATASET")
print("="*30)

# Generate predictions for the entire dataset
X_all_scaled = X_scaled
y_pred_all = best_xgb_model.predict(X_all_scaled)
y_pred_proba_all = best_xgb_model.predict_proba(X_all_scaled)[:, 1]

# Create export dataset
export_data = telecom_data.copy()
export_data['churn_probability'] = y_pred_proba_all
export_data['predicted_churn'] = y_pred_all
export_data['risk_level'] = pd.cut(y_pred_proba_all, 
                                  bins=[0, 0.3, 0.7, 1.0], 
                                  labels=['Low', 'Medium', 'High'])

# Add model confidence and feature insights
export_data['model_confidence'] = np.where(
    (y_pred_proba_all < 0.2) | (y_pred_proba_all > 0.8), 'High', 
    np.where((y_pred_proba_all < 0.35) | (y_pred_proba_all > 0.65), 'Medium', 'Low')
)

# Add business metrics
export_data['ltv_at_risk'] = export_data['monthly_charges'] * 24 * export_data['churn_probability']
export_data['intervention_priority'] = export_data['churn_probability'].rank(ascending=False, method='dense')

print(f"Export dataset shape: {export_data.shape}")
print(f"Columns in export dataset: {list(export_data.columns)}")
print()

# Summary statistics for export
print("EXPORT DATASET SUMMARY")
print("-" * 25)
print("Risk Level Distribution:")
print(export_data['risk_level'].value_counts())
print()
print("Model Confidence Distribution:")
print(export_data['model_confidence'].value_counts())
print()

# Export to CSV
export_filename = 'telecom_churn_predictions.csv'
export_data.to_csv(export_filename, index=False)
print(f"Dataset exported to: {export_filename}")
print(f"File size: {len(export_data)} rows, {len(export_data.columns)} columns")
print()

# Create summary report
summary_stats = {
    'total_customers': len(export_data),
    'overall_churn_rate': export_data['churn'].mean(),
    'avg_churn_probability': export_data['churn_probability'].mean(),
    'high_risk_customers': (export_data['risk_level'] == 'High').sum(),
    'high_confidence_predictions': (export_data['model_confidence'] == 'High').sum(),
    'total_monthly_revenue': export_data['monthly_charges'].sum(),
    'revenue_at_risk': export_data['ltv_at_risk'].sum(),
    'model_accuracy': xgb_accuracy,
    'model_auc': xgb_auc
}

print("FINAL SUMMARY REPORT")
print("="*30)
for key, value in summary_stats.items():
    if isinstance(value, float):
        if 'rate' in key or 'probability' in key or 'accuracy' in key or 'auc' in key:
            print(f"{key.replace('_', ' ').title()}: {value:.1%}")
        else:
            print(f"{key.replace('_', ' ').title()}: ${value:,.2f}")
    else:
        print(f"{key.replace('_', ' ').title()}: {value:,}")
        
print()
print("DASHBOARD INTEGRATION READY")
print("The exported CSV file contains all necessary fields for:")
print("- Customer risk segmentation")
print("- Intervention prioritization") 
print("- Revenue impact analysis")
print("- Model performance monitoring")

# Display sample of high-risk customers for verification
print("\nSAMPLE HIGH-RISK CUSTOMERS:")
high_risk_sample = export_data[export_data['risk_level'] == 'High'][
    ['customer_id', 'churn_probability', 'monthly_charges', 'contract_type', 
     'tenure_months', 'ltv_at_risk']].head(10)
print(high_risk_sample.to_string(index=False))
